In [ ]:
import pandas as pd 
import pickle
import datetime
import re
import networkx as nx
import numpy as np
from tqdm import tqdm

# Atrial fibrillation (AF) patients on warfarin

In [ ]:
with open('/projects/ROCKET AF/1A/warfarin dataextraction/data/first_mention_warfarin.pickle', 'rb') as f:
    warfarin = pickle.load(f)

In [ ]:
len(warfarin)

In [ ]:
df = pd.DataFrame(list(warfarin.items()), columns=['client_idcode', 'first_mention_warfarin']) # This code converts a dictionary (warfarin) into a Pandas DataFrame with two columns named 'client_idcode' and 'first_mention_warfarin'
df.shape

In [ ]:
df.head()

In [ ]:
ids = set(df['client_idcode'])
len(ids)

# AF patients on daily aspirin dose greater than 100mg

In [ ]:
aspirin = pd.read_csv('/projects/ROCKET AF/data/aspirin.csv')

In [ ]:
aspirin.head()

In [ ]:
aspirin.shape

In [ ]:
aspirin['first_warfarin_prescription'] = aspirin['client_idcode'].map(warfarin)

In [ ]:
aspirin.head()

In [ ]:
aspirin = aspirin[aspirin['first_warfarin_prescription'].notna()]
aspirin.shape

In [ ]:
aspirin['client_idcode'].nunique()

In [ ]:
aspirin = aspirin[['client_idcode', 'order_name', 'order_arrivaldtm', 'order_summaryline', 'first_warfarin_prescription' ]]

In [ ]:
aspirin.head()

In [ ]:
aspirin['order_summaryline'].nunique()

In [ ]:
aspirin['order_summaryline'] = aspirin['order_summaryline'].str.lower()

In [ ]:
aspirin.head()

In [ ]:
aspirin['order_name'].unique()

In [ ]:
aspirin['dose'] = aspirin['order_summaryline'].str.lower().str.slice(stop=3).str.strip()

In [ ]:
aspirin.head()

In [ ]:
aspirin['dose'].unique()

In [ ]:
aspirin.loc[aspirin['dose'] == '[no']

In [ ]:
aspirin.loc[aspirin['dose'] == '(pr']

In [ ]:
aspirin.loc[aspirin['dose'] == '[pr']

In [ ]:
for i,row in tqdm(aspirin.iterrows()):
    if '[non formulary]' not in aspirin.loc[i,'order_summaryline'] and '(prescribed in icu)' not in aspirin.loc[i,'order_summaryline'] and '[pre-pack]' not in aspirin.loc[i,'order_summaryline'] and '[non-formulary]' not in aspirin.loc[i,'order_summaryline'] :
        aspirin.at[i, 'dose'] = aspirin.loc[i,'order_summaryline'].split('mg')[0].strip()[:3]
    else:
        aspirin.at[i,'dose'] = np.nan       

In [ ]:
aspirin.head()

In [ ]:
aspirin['dose'].unique()

In [ ]:
aspirin['dose'].isna().sum()

In [ ]:
for i,row in tqdm(aspirin.iterrows()):
    if '[non formulary]' in aspirin.loc[i,'order_summaryline']:
        aspirin.loc[i,'dose'] = aspirin.loc[i,'order_summaryline'].split('[non formulary]')[1].strip()[:3].strip()

In [ ]:
aspirin['dose'].isna().sum()

In [ ]:
aspirin['dose'].unique()

In [ ]:
aspirin.loc[aspirin['dose'] == '1 t']

In [ ]:
aspirin.loc[71272, 'order_summaryline']

In [ ]:
aspirin.loc[aspirin['dose'] == '1 t', 'dose'] = '75'

In [ ]:
aspirin.loc[71272]

In [ ]:
aspirin.loc[aspirin['dose'] == '0']

In [ ]:
aspirin['dose'].unique()

In [ ]:
for i,row in tqdm(aspirin.iterrows()):
    if '[non-formulary]' in aspirin.loc[i,'order_summaryline']:
        aspirin.loc[i,'dose'] = aspirin.loc[i,'order_summaryline'].split('[non-formulary]')[1].strip()[:3].strip()

In [ ]:
aspirin['dose'].isna().sum()

In [ ]:
aspirin['dose'].unique()

In [ ]:
aspirin.loc[aspirin['dose'] == '57']

In [ ]:
aspirin.loc[aspirin['dose'] == '1 t', 'dose'] = '300'

In [ ]:
aspirin['dose'].unique()

In [ ]:
for i,row in tqdm(aspirin.iterrows()):
    if '(prescribed in icu)' in aspirin.loc[i,'order_summaryline']:
        aspirin.loc[i,'dose'] = aspirin.loc[i,'order_summaryline'].split('(prescribed in icu)')[1].strip()[:3].strip()

In [ ]:
aspirin['dose'].unique()

In [ ]:
aspirin['dose'].isna().sum()

In [ ]:
for i,row in tqdm(aspirin.iterrows()):
    if '[pre-pack]' in aspirin.loc[i,'order_summaryline']:
        aspirin.loc[i,'dose'] = aspirin.loc[i,'order_summaryline'].split('[pre-pack]')[1].strip()[:3].strip()

In [ ]:
aspirin['dose'].unique()

In [ ]:
aspirin['dose'].isna().sum()

In [ ]:
aspirin.info()

In [ ]:
aspirin['dose'] = aspirin['dose'].astype(int)

In [ ]:
aspirin.info()

In [ ]:
aspirin.head()

In [ ]:
aspirin['dose'].unique()

In [ ]:
aspirin = aspirin.loc[aspirin['dose'] > 100]

In [ ]:
aspirin.head()

In [ ]:
aspirin.shape

In [ ]:
aspirin.head()

In [ ]:
aspirin['order_arrivaldtm'] =  pd.to_datetime(aspirin['order_arrivaldtm'], utc=True).dt.date 

In [ ]:
aspirin.head()

In [ ]:
aspirin['difference'] = pd.to_datetime(aspirin['first_warfarin_prescription']) - pd.to_datetime(aspirin['order_arrivaldtm'])

In [ ]:
aspirin.head()

In [ ]:
min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
t1 = aspirin['difference'] >= min_time 
t2 = aspirin['difference'] <= max_time
in_window = t1 & t2 
aspirin['in_window'] = in_window

In [ ]:
aspirin = aspirin.loc[aspirin['in_window'] == True]

In [ ]:
aspirin.head()

In [ ]:
aspirin.shape

In [ ]:
ids = set(aspirin['client_idcode'])
len(ids)

In [ ]:
with open('/projects/ROCKET AF/1A/warfarin dataextraction/data/aspirin_>_100.pickle', 'wb') as f:
    pickle.dump(ids,f)